# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv ('City.csv')
city_data


,City Name,Latitude,Longitude,Temperature,Wind Speed,Cloudiness,Humidity,Country,Date
0,sinnamary,5.3833,-52.9500,80.92,9.48,99,80,GF,1650918158
1,dingle,10.9995,122.6711,78.01,6.93,48,88,PH,1650918158
2,rikitea,-23.1203,-134.9692,77.16,15.70,76,72,PF,1650917936
3,mezen,65.8522,44.2400,33.21,5.01,100,90,RU,1650918159
4,marawi,7.9986,124.2928,69.93,2.98,100,91,PH,1650918159
...,...,...,...,...,...,...,...,...,...
559,adre,13.4667,22.2000,85.53,8.63,100,29,TD,1650918381
560,sabang,5.8933,95.3214,82.27,18.28,100,78,ID,1650918346
561,hlobyne,49.3878,33.2596,60.10,11.97,100,68,UA,1650918382
562,nisia floresta,-6.0911,-35.2086,84.58,9.60,35,81,BR,1650918383


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [39]:
locations = city_data[["Latitude", "Longitude"]]


fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=city_data["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
ideal_weather = city_data.loc[(city_data['Temperature'] >= 70) & (city_data['Temperature'] <= 80)]  
ideal_weather = city_data.loc[(city_data["Temperature"] >= 70) & (city_data["Temperature"] <= 80) & (city_data['Wind Speed']<= 10) & (city_data['Cloudiness'] == 0)]


ideal_weather 

clean_data= ideal_weather .dropna(axis=0)

clean_data


,City Name,Latitude,Longitude,Temperature,Wind Speed,Cloudiness,Humidity,Country,Date
75,carnarvon,-24.8667,113.6333,71.67,6.91,0,78,AU,1650918190
103,hilton head island,32.2163,-80.7526,79.07,6.91,0,66,US,1650917949
122,castro,-24.7911,-50.0119,71.89,3.49,0,68,BR,1650918207
426,saint george,37.1041,-113.5841,75.56,5.75,0,12,US,1650918184
527,ramanuj ganj,23.8000,83.7000,78.12,4.41,0,15,IN,1650918368


In [43]:
hotel_df = clean_data
hotel_df

,City Name,Latitude,Longitude,Temperature,Wind Speed,Cloudiness,Humidity,Country,Date
75,carnarvon,-24.8667,113.6333,71.67,6.91,0,78,AU,1650918190
103,hilton head island,32.2163,-80.7526,79.07,6.91,0,66,US,1650917949
122,castro,-24.7911,-50.0119,71.89,3.49,0,68,BR,1650918207
426,saint george,37.1041,-113.5841,75.56,5.75,0,12,US,1650918184
527,ramanuj ganj,23.8000,83.7000,78.12,4.41,0,15,IN,1650918368


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
hotel_df['Hotel Name'] = ""

hotel_df

,City Name,Latitude,Longitude,Temperature,Wind Speed,Cloudiness,Humidity,Country,Date,Hotel Name
75,carnarvon,-24.8667,113.6333,71.67,6.91,0,78,AU,1650918190,
103,hilton head island,32.2163,-80.7526,79.07,6.91,0,66,US,1650917949,
122,castro,-24.7911,-50.0119,71.89,3.49,0,68,BR,1650918207,
426,saint george,37.1041,-113.5841,75.56,5.75,0,12,US,1650918184,
527,ramanuj ganj,23.8000,83.7000,78.12,4.41,0,15,IN,1650918368,


In [45]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row['Latitude']
    lon = row['Longitude']
    
    params['location']= f'{lat},{lon}'


    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City Name']}.")
    
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {hotel_df} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
 
    params


Retrieving Results for Index 75: carnarvon.
Closest               City Name  Latitude  Longitude  Temperature  Wind Speed  \
75            carnarvon  -24.8667   113.6333        71.67        6.91   
103  hilton head island   32.2163   -80.7526        79.07        6.91   
122              castro  -24.7911   -50.0119        71.89        3.49   
426        saint george   37.1041  -113.5841        75.56        5.75   
527        ramanuj ganj   23.8000    83.7000        78.12        4.41   

     Cloudiness  Humidity Country        Date Hotel Name  
75            0        78      AU  1650918190             
103           0        66      US  1650917949             
122           0        68      BR  1650918207             
426           0        12      US  1650918184             
527           0        15      IN  1650918368              hotel is Hospitality Carnarvon.
------------
Retrieving Results for Index 103: hilton head island.
Closest               City Name  Latitude  Longitude  Te

In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd>Hospitality Carnarvon</dd>\n<dt>City Name</dt><dd>carnarvon</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Disney’s Hilton Head Island Resort</dd>\n<dt>City Name</dt><dd>hilton head island</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>CHACARA BAILLY</dd>\n<dt>City Name</dt><dd>castro</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Best Western Plus Abbey Inn</dd>\n<dt>City Name</dt><dd>saint george</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>HOTEL SAI SIDDHI</dd>\n<dt>City Name</dt><dd>ramanuj ganj</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n']

In [49]:
# Add marker layer ontop of heat map

# Display figure
heat_layer = gmaps.heatmap_layer(city_data[["Latitude", "Longitude"]], weights=city_data["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)

marker_layer = gmaps.marker_layer(
    locations, 
    info_box_content = hotel_info
)

fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig





Figure(layout=FigureLayout(height='420px'))